In [1]:
from interval_search import doubling_search
import math
from nbmetalog import nbmetalog as nbm
import numpy as np
import pandas as pd
import random
from scipy import stats
import sympy
import typing

random.seed(1)


In [2]:
nbm.print_metadata()


context: ci
hostname: 5c076fe67bab
interpreter: 3.8.12 (default, Jan 15 2022, 18:39:47)  [GCC 7.5.0]
nbcellexec: 2
nbname: maximum_likelihood_popsize_estimator_credible_interval
nbpath: /opt/hereditary-stratigraph-concept/binder/popsize/maximum_likelihood_popsize_estimator_credible_interval.ipynb
revision: null
session: 1175dcba-2d91-402b-81bc-4243802a9d9b
timestamp: 2022-10-23T03:57:45Z00:00


IPython==7.16.1
keyname==0.4.1
yaml==5.3.1
nbmetalog==0.2.6
numpy==1.21.5
pandas==1.1.2
scipy==1.5.4
sympy==1.5.1
re==2.2.1
ipython_genutils==0.2.0
logging==0.5.1.2
zmq==22.3.0
json==2.0.9
ipykernel==5.5.3


# Goal

From [maximum_likelihood_estimator.ipynb](maximum_likelihood_estimator.ipynb) we have the maximum likelihood estimator for $n$ given $k$ observations of fixed gene magnitude $x_1, x_2, ... x_k$ as

$\hat{n}_\mathrm{mle} = -\frac{k}{\sum_{i=1}^k \log( x_i )}$.

In order to better understand our estimate, we should develop an expression for uncertainty related to the estimate.


# Likelihood & Credibility

We can express probability that the true value of $n$ falls within a range as the fraction of total likelihood that falls within that range.
This constitutes a Bayesian "credible interval," which differs subtly from a (frequentist) confidence interval [(Porter, 1996)](porder1996interval).
Note that this assumes a uniform prior for $n$ over $\mathbb{R}_{\ge 0}$.


# Normalization Factor $L$

To begin, calculate total likelihood $L$ by integrating over the domain of $n$.
(For simplicity, we treat $n$ as continuous rather than discrete.)

$\begin{align*}
L
&= \int_{0}^{\infty} \mathcal{L}(n) \, \mathrm{d}n \\
&= \int_{0}^{\infty} \exp(\log\mathcal{L}(n)) \, \mathrm{d}n \\
&= \int_{0}^{\infty} \exp\Big((n-1) \sum_{i=1}^k \log( x_i ) + k \log(n)\Big) \, \mathrm{d}n \\
&= \int_{0}^{\infty} \exp\Big((n-1) \sum_{i=1}^k \log( x_i )\Big) \exp\Big(k \log(n)\Big) \, \mathrm{d}n \\
&= \int_{0}^{\infty} \exp\Big((n-1) \sum_{i=1}^k \log( x_i )\Big) \exp\Big( \log(n^k)\Big) \, \mathrm{d}n \\
&= \int_{0}^{\infty} \exp\Big((n-1) \sum_{i=1}^k \log( x_i )\Big) n^k \, \mathrm{d}n.
\end{align*}$

Let's simplify our expression by substituting $\sum_{i=1}^k \log( x_i )$ as $v$,

$\begin{align*}
L
&= \int_{0}^{\infty} \exp\Big((n-1) v\Big) n^k \, \mathrm{d}n
\end{align*}$

and evaluate this integral with the help of computer algebra.


In [3]:
# specify variables with assumptions for domain
n = sympy.Symbol('n', positive=True, real=True,)
k = sympy.Symbol('k', integer=True, positive=True, real=True,)
v = sympy.Symbol('v', negative=True, real=True,)

likelihood = sympy.exp( (n-1) * v ) * n**k

# pretty print, does the expression look right?
sympy.Integral(likelihood, (n, 0, sympy.oo))


Integral(n**k*exp(v*(n - 1)), (n, 0, oo))

In [4]:
# perform integration
likelihood_integrated_over_domain = sympy.integrate(
    likelihood,
    (n, 0, sympy.oo)
)
likelihood_integrated_over_domain


-(-v)**(-k)*exp(-v)*gamma(k + 1)/v

Simplifying and substituting,

$\begin{align*}
L
&= -\frac{(-v)^{-k} e^{-v} \Gamma(k+1)}{v}\\
&= -\frac{-v(-v)^{-k-1} e^{-v} \Gamma(k+1)}{v}\\
&= (-v)^{-k-1} e^{-v} \Gamma(k+1).
\end{align*}$


# Credibility Within Factor of MLE

How much likelihood $L_f$ falls within a factor $f$ of our maximum likelihood estimate?
Integrating likelihood $\mathcal{L}$ between $\hat{n}_\mathrm{mle}/f$ and $f\hat{n}_\mathrm{mle}$,

$\begin{align*}
L_f
&= \int_{\hat{n}_\mathrm{mle}/f}^{f\hat{n}_\mathrm{mle}} \mathcal{L}(n) \, \mathrm{d}n\\
&= \int_{\frac{1}{f}\frac{k}{-\sum_{i=1}^k \log( x_i )}}^{f\frac{k}{-\sum_{i=1}^k \log( x_i )}} \exp\Big((n-1) v\Big) n^k \, \mathrm{d}n.
\end{align*}$

Once more simplifying this expression by substituting $\sum_{i=1}^k \log( x_i )$ as $v$,

$\begin{align*}
L_f
&= \int_{\frac{k}{-fv}}^{\frac{fk}{-v}} \exp\Big((n-1) v\Big) n^k \, \mathrm{d}n.
\end{align*}$

we will again rely on computer algebra to evaluate this integral.


In [5]:
# specify variables with assumptions for domain
f = sympy.Symbol('f', positive=True, real=True,)

factor_interval_lb = - k / (f * v)
factor_interval_ub = - f * k / v

# pretty print, does the expression look right?
sympy.Integral(
    likelihood,
    (n, factor_interval_lb, factor_interval_ub,),
)


Integral(n**k*exp(v*(n - 1)), (n, -k/(f*v), -f*k/v))

In [6]:
# perform integration
likelihood_integrated_over_credible_interval = sympy.integrate(
    likelihood,
    (n, factor_interval_lb, factor_interval_ub,),
)
likelihood_integrated_over_credible_interval.simplify()


v**(-k - 1)*(lowergamma(k + 1, k/f) - lowergamma(k + 1, f*k))*exp(I*pi*k - v)

We can now solve for the credibilty contained within the credible interval by taking the ratio of integrated likelihoods.


In [7]:
credibility = (
    likelihood_integrated_over_credible_interval
    / likelihood_integrated_over_domain
).simplify()
credibility


(-lowergamma(k + 1, k/f) + lowergamma(k + 1, f*k))/factorial(k)

# What Interval Factor Is Required to Capture a Target Credibility?

Unfortunately, there is no easy rearrangement to analytically express the number of required independent observations $k$ in terms of the required credibility and the desired creditiblity interval width factor $f$.


In [8]:
# trying to solve for an expression that gives f
# which will yield 95% credibility fails
try:
    sympy.solve(sympy.Eq( credibility, 0.95 ), k)
except NotImplementedError as e:
    print(e)


multiple generators [factorial(k), lowergamma(k + 1, f*k), lowergamma(k + 1, k/f)]
No algorithms are implemented to solve equation (-lowergamma(k + 1, k/f) + lowergamma(k + 1, f*k))/factorial(k) - 19/20


However, we can nonetheless efficiently compute the number of required independent observations $k$ to capture a target credibility within a particular factor of the MLE by means of exponential search.


In [9]:
def num_observations_required_for_credibility(
    target_credibility: float,
    interval_factor: float,
) -> int:
    """
    Find the minimum number of observations required for a threshold amount of credibility to be contained within a factor of the MLE estimate for
    population size.

    Parameters
    ----------
    target_credibility : float
        What credibility is required for the credible interval?
    interval_factor : float
        What should the credible interval bounds be, as a factor of the MLE estimate?
        For instance, 2 would indicate the credible interval should span from half of the MLE estimate to twice the MLE estimate.
        Corresponds to $f$ in symbolic scratchwork elsewhere.
    upper_bound : int
        Upper bound for the binary search, inclusive.
    """

    assert 0.0 <= target_credibility <= 1.0
    assert interval_factor > 1.0

    predicate = lambda k_: credibility.evalf(
        subs={
            k : k_,
            f : interval_factor,
        },
    ) >= target_credibility

    return doubling_search( predicate )


How many independent observations are required to capture 95% of credibility for different credible interval size factors?


In [10]:
interval_factors = (
    1.1,
    1.5,
    2.0,
    3.0,
    4.0,
    6.0,
)

atleast95cred_intervals_df = pd.DataFrame.from_records([
    {
        'Factor of MLE with 95% Credibility' : interval_factor,
        'Num Independent Observations Required' : num_observations_required_for_credibility(0.95, interval_factor),
    }
for interval_factor in interval_factors])

atleast95cred_intervals_df


,Factor of MLE with 95% Credibility,Num Independent Observations Required
0,1.1,423
1,1.5,23
2,2.0,8
3,3.0,3
4,4.0,2
5,6.0,1


# Simulated Experiments


How do the derived maximum likelihood estimator and credibility intervals perform on simulated experiments?


In [11]:
def sample_observations(true_popsize: int, num_observations: int) -> typing.List[float]:
    """Simulate sampling the largest gene from within a population of `true_popsize` `num_observations` times."""

    return [
        max(random.random() for __ in range(true_popsize))
        for __ in range(num_observations)
    ]

def estimate_popsize(observations: typing.List[float]) -> float:
    """Use maximum likelihood estimator to estimate underlying population size given `observations`."""

    return -len(observations) / sum(math.log(o) for o in observations)

def sample_popsize_estimate(true_popsize: int, num_observations: int) -> float:
    """Generate sampled largest genes from `true_popsize` population
    and then use maximum likelihood estimator to estimate `true_popsize`."""

    return estimate_popsize(sample_observations(true_popsize, num_observations))


In [12]:
# simulate gene drive within populations and then subsequent estimates of population size from magnitude of fixed genes
records = []
for true_popsize in 10, 1000:
    for __, row in atleast95cred_intervals_df.iterrows():
        sampled_estimates = [
            sample_popsize_estimate(
                true_popsize,
                int(row['Num Independent Observations Required']),
            )
            for __ in range(200)
        ]

        f = row['Factor of MLE with 95% Credibility']
        num_estimates_within_credible_interval = sum(
            true_popsize / f <= est <= true_popsize * f
            for est in sampled_estimates
        )

        records.append({
            **row,
            **{
                'True Population Size' : true_popsize,
                'Mean Estimated Population Size' : np.mean(sampled_estimates),
                'Median Estimated Population Size' : np.median(sampled_estimates),
                'Mean Normalized Error' : np.mean([abs(est - true_popsize) for est in sampled_estimates]) / true_popsize,
                'Normalized Mean Estimate' : np.mean(sampled_estimates) / true_popsize,
                'Normalized Median Estimate' : np.median(sampled_estimates) / true_popsize,
                'Factor of MLE with 95% Credibility' : f,
                'Fraction Estimates within Credible Interval'
                    : num_estimates_within_credible_interval / len(sampled_estimates),
                'p As Many Estimates Outside Credible Interval'
                    : stats.binom.cdf(num_estimates_within_credible_interval, len(sampled_estimates), 0.95),
            },
        })


In [13]:
res_df = pd.DataFrame.from_records(records)
res_df.round(5)


,Factor of MLE with 95% Credibility,Num Independent Observations Required,True Population Size,Mean Estimated Population Size,Median Estimated Population Size,Mean Normalized Error,Normalized Mean Estimate,Normalized Median Estimate,Fraction Estimates within Credible Interval,p As Many Estimates Outside Credible Interval
0,1.1,423.0,10,10.00820,9.96765,0.04005,1.00082,0.99676,0.940,0.30024
1,1.5,23.0,10,10.57572,10.21900,0.18420,1.05757,1.02190,0.940,0.30024
2,2.0,8.0,10,11.08411,10.44453,0.32556,1.10841,1.04445,0.930,0.12989
3,3.0,3.0,10,15.63577,11.67543,0.76309,1.56358,1.16754,0.895,0.00116
4,4.0,2.0,10,17.61099,11.11620,1.02256,1.76110,1.11162,0.915,0.02380
5,6.0,1.0,10,53.56253,13.09602,4.71834,5.35625,1.30960,0.860,0.00000
6,1.1,423.0,1000,1005.32745,1005.82841,0.03704,1.00533,1.00583,0.935,0.20352
7,1.5,23.0,1000,1055.09799,1011.89282,0.18717,1.05510,1.01189,0.945,0.41693
8,2.0,8.0,1000,1153.30489,1048.24615,0.32236,1.15330,1.04825,0.935,0.20352
9,3.0,3.0,1000,1317.26717,1107.06680,0.60986,1.31727,1.10707,0.930,0.12989


The maximum likelihood population size estimator $\hat{n}_{\mathrm{MLE}}$ appears to be biased towards overestimation, especially for small numbers of independent observations.
For one independent observation, 95% credibility appears to translate to about 80% confidence.

Perhaps numerical simulation paired with binomial confidence intervals could be used to produce (loose) confidence intervals for this estimation method.


# Result

We derived an expression for the credibilty contained within a factor of the maximum likelihood estimate $\hat{n}_\mathrm{mle}$,

$$\frac{- \gamma(k + 1, \frac{k}{f}) + \gamma(k + 1, f k)}{k!}.$$

By its form, the credibility contained within a factor of maximum likelihood estimate $\hat{n}_\mathrm{mle}$ remains constant across population sizes $n$.
Numerical simulation showed that $\hat{n}_\mathrm{mle}$ is biased towards overestimation, although this bias decreases with increasing independent observations $k$.
Numerical simulation also showed a clear nonequivalence between confidence and credibility, although this dissipated with larger $k$.


# References

<a
   id="porter1996interval"
   href="https://www.jstor.org/stable/2985006">
Porter, Frank. "Interval estimation using the likelihood function." Nuclear Instruments and Methods in Physics Research Section A: Accelerators, Spectrometers, Detectors and Associated Equipment 368.3 (1996): 793-803.
</a>
